In [1]:
import re
import pandas as pd
import numpy as np
# from mlutils import dataset, connector
import scipy.stats
from scipy.stats import *
from sklearn.preprocessing import StandardScaler
import math
import matplotlib.pyplot as plt
import warnings
import statsmodels.api as sm 
import seaborn as sns
import pylab as py 
warnings.filterwarnings('ignore')
%matplotlib inline

In [25]:
def standarise(y,pct,pct_lower):
    sc = StandardScaler() 
    y.sort()
    len_y = len(y)
    y = y[int(pct_lower * len_y):int(len_y * pct)]
    len_y = len(y)
    yy=([[x] for x in y])
    sc.fit(yy)
    y_std =sc.transform(yy)
    y_std = y_std.flatten()
    return y_std,len_y,y

In [3]:
def fit_distribution(column,pct,pct_lower):
    # Set up list of candidate distributions to use
    # See https://docs.scipy.org/doc/scipy/reference/stats.html for more
    y_std,size,y_org = standarise(column,pct,pct_lower)
    dist_names = ['weibull_min','norm','weibull_max','beta',
                 'invgauss','uniform','gamma','expon', 'lognorm','pearson3','triang']

    chi_square_statistics = []
    # 11 bins
    percentile_bins = np.linspace(0,100,11)
    percentile_cutoffs = np.percentile(y_std, percentile_bins)
    observed_frequency, bins = (np.histogram(y_std, bins=percentile_cutoffs))
    cum_observed_frequency = np.cumsum(observed_frequency)

    # Loop through candidate distributions

    for distribution in dist_names:
        # Set up distribution and get fitted distribution parameters
        dist = getattr(scipy.stats, distribution)
        param = dist.fit(y_std)
        print("{}\n{}\n".format(dist, param))
      

        # Get expected counts in percentile bins
        # cdf of fitted sistrinution across bins
        cdf_fitted = dist.cdf(percentile_cutoffs, *param)
        expected_frequency = []
        for bin in range(len(percentile_bins)-1):
            expected_cdf_area = cdf_fitted[bin+1] - cdf_fitted[bin]
            expected_frequency.append(expected_cdf_area)

        # Chi-square Statistics
        expected_frequency = np.array(expected_frequency) * size
        cum_expected_frequency = np.cumsum(expected_frequency)
        ss = round(sum (((cum_expected_frequency - cum_observed_frequency) ** 2) / cum_observed_frequency),0)
        chi_square_statistics.append(ss)


    #Sort by minimum ch-square statistics
    results = pd.DataFrame()
    results['Distribution'] = dist_names
    results['chi_square'] = chi_square_statistics
    results.sort_values(['chi_square'], inplace=True)


    print('\nDistributions listed by Betterment of fit:')
    print('............................................')
    print(results)

In [23]:
inter_arrival=[95.75696400000015, 0.0, 150.49551800000336, 50.88580199999706, 181.88004699999874, 144.7160600000061, 416.8790289999961, 57.81027300000278, 58.63326699999743, 167.2136180000016, 71.48144099999627, 0.0, 60.36459200000536, 185.62336800000048, 87.93199099999765, 25.08961299999646, 10.442488000000594, 22.57304600000498, 93.99067799999466, 8.591490999999223, 43.19866400000319, 0.0, 100.89947699999902, 399.55726400000276, 79.88819199999853, 389.36331999999675, 40.626239000004716, 223.3120129999952, 0.0, 693.9347610000041, 27.517260000000533, 192.88420600000245, 0.0, 0.0, 67.46234499999991, 6.79896899999585, 259.817541000004, 112.47898499999428, 151.8366090000054, 24.702197999999044, 191.1176869999981, 310.828247999998, 371.9643389999983, 95.70794900000328, 309.32546300000104, 43.14413699999568, 76.2420740000016, 3.3892200000045705, 30.065223999998125, 395.9861660000024, 80.79625699999451, 48.222852999999304, 532.2801240000044, 0.0, 8.167611999997462, 83.79593300000124, 35.92706999999791, 3.9148260000001756, 210.33403399999952, 193.63486900000134, 8.40545099999872, 304.20118699999875, 89.4864000000016, 48.678413000001456, 0.0, 54.58960500000103, 22.941677000002528, 58.6644629999937, 63.26886500000546, 57.647673999999824, 48.520731999997224, 128.49727500000154, 31.44557299999724, 51.14923600000475, 0.0, 37.446281999997154, 55.07967400000052, 42.868302000002586, 181.38228899999376, 222.23432400000456, 9.8686699999962, 35.16718799999944, 84.58632200000284, 96.32021199999872, 94.74560000000201, 2.562307000000146, 74.45805200000177, 1.3109239999976126, 4.311791000000085, 16.428697000003012, 25.373225999996066, 2.6253169999981765, 18.984700000000885, 541.0775429999994, 7.90461000000505, 38.03269, 252.07554600000003, 40.26919999999518, 5.69300099999964, 6.679253000002063, 3.4218540000001667, 22.88166800000181, 40.354059999997844, 0.0, 57.39722900000197, 120.17203900000095, 219.95079500000065, 41.56658299999981, 105.03207800000018, 61.9186469999986, 8.242770000000746, 67.11078300000008, 202.04033499999787, 0.0, 308.38406300000133, 0.0, 94.67932699999801, 7.787994000002072, 11.809406000000308, 229.31902400000035, 162.83698199999708, 30.34602200000154, 41.36112800000046, 19.90747900000133, 377.501014999998, 297.34621400000105, 0.0, 64.14672000000064, 40.9649759999993, 479.6169160000027, 106.61276499999803, 400.34965699999884, 9.118954999998095, 388.9302650000027, 281.84553700000106, 11.811770000000251, 86.35472699999809, 170.36585499999637, 368.25823500000115, 56.29380900000251, 326.0515579999992, 263.04673000000184, 68.00727399999596, 168.02084900000045, 38.328661999999895, 72.27102500000183, 113.65360499999952, 350.56501299999945, 50.70281700000487, 170.30298399999447, 0.0, 204.83524300000136, 151.04338300000381, 55.47626799999853, 88.19936400000006, 44.55937499999709, 189.53405000000203, 129.12911399999575, 135.53945700000622, 138.95980299999792, 119.46242699999857, 16.35529800000222, 819.0545569999958, 30.811034000005748, 19.537039999995613, 86.99965900000097, 81.59014800000296, 56.72226399999636, 150.11053399999946, 2.354307000001427, 34.107826999999816, 41.193098000003374, 43.874408999996376, 0.0, 289.0838920000024, 38.29254099999525, 135.5868780000019, 4.328515000001062, 91.37027300000045, 195.1295969999992, 92.39873200000147, 2.915443999998388, 114.38199899999745, 151.26663199999894, 60.57393000000593, 219.08408199999394, 86.566104000005, 10.004604000001564, 9.31161499999871, 264.209004999997, 17.568277999998827, 26.010418999998365, 105.32623700000113, 98.66475999999966, 104.6809659999999, 58.57393000000593, 287.59638899999845, 4.219819999998435, 91.51745700000174, 195.08959699999832, 209.73617499999818, 151.26663199999894, 60.57393000000593, 219.08408199999394, 86.566104000005, 10.004604000001564, 8.960761999995157, 376.853654999999, 39.75795500000095, 493.7354600000035, 163.7810959999988, 339.5185599999968, 37.688348000003316, 337.80483300000196, 38.29254099999525, 249.6798259999996, 1.7869040000005043, 38.94630100000359, 86.19225300000107, 259.50615399999515, 262.66851500000485, 141.83297799999855, 190.8326339999985, 317.9756419999976, 39.75795500000095, 383.57698399999936, 361.3628749999989, 42.870639000000665, 209.2246180000002, 37.688348000003316, 67.54241799999727, 349.16788700000325, 5.039233999996213, 49.000222000002395, 56.18211899999733, 96.08135900000343, 271.8981300000014, 239.53690600000118, 13.675541999997222, 310.8881790000014, 28.0574689999994, 7.649315999995451, 215.49761399999988, 42.870639000000665, 141.8243350000048, 2.915443999998388, 114.38199899999745, 322.5658850000036, 176.90876600000047, 4.319299999995565, 274.4577170000048, 71.9602539999978, 247.7638719999959, 90.20550000000367, 451.00456100000156, 114.05595599999651, 114.91342299999815, 381.91421000000264, 56.298063000002, 46.376116999999795, 35.18130799999926, 15.764343999995617, 473.7119590000002, 89.65624500000558, 138.16709299999638, 0.0, 83.5930440000011, 99.74502899999788, 88.53817300000082, 0.0, 11.966386000000057, 14.160907999998017, 393.76891000000614, 127.81668199999694, 72.00998000000254, 205.16540800000075, 0.0, 159.2834010000006, 104.22680499999842, 111.74238999999943, 5.6391419999999925, 198.0794659999956, 223.96728600000642, 16.869025999993028, 192.2833030000038, 80.7717049999992, 128.45169600000372, 186.52032699999836, 0.0, 0.0, 0.0, 151.23454199999833, 20.973996000000625, 178.08179599999858, 0.0, 32.31792900000437, 0.0, 0.0, 359.8434599999964, 472.1020290000015, 5.833896999996796, 205.39027999999962, 152.4005860000034, 0.0, 212.73154100000102, 63.03157099999953, 146.0253259999954, 59.36528900000121, 73.5790960000013, 2.101701999999932, 40.920867000000726, 20.95555299999978, 189.3351229999971, 29.086014000000432, 142.79736000000412, 8.378186000001733, 202.16930599999614, 29.056664000003366, 210.80657199999405, 72.69340600000578]

In [27]:
fit_distribution(inter_arrival,0.99,0.01)

(0.7015384049768638, -0.9894365309023049, 1.2006676444778561)

(0.0, 1.0)

(0.7019179793807708, 3.26754168282175, 1.2352709611338757)

(0.5869141736933967, 2.0230692974358018, -0.9894365309023049, 5.611707278987666)

(1.3774125862615234, -1.133748597072893, 0.8230934376760495)

(-0.9894365309023048, 4.256978213724054)

(0.8667586476420364, -0.989436530902305, 0.8160179858473148)

(-0.9894365309023048, 0.9894365309023048)

(1.2285098631304354, -1.040346141653061, 0.5785087400178923)

(2.416641101443037, -0.007126039833642914, 1.1869459535476108)

(1.2422294892673428e-07, -0.9894371622132272, 4.384475250138744)


Distributions listed by Betterment of fit:
............................................
   Distribution  chi_square
9      pearson3         9.0
0   weibull_min        13.0
3          beta        22.0
8       lognorm        26.0
4      invgauss        29.0
7         expon        30.0
6         gamma        48.0
10       triang       210.0
1          norm       333.0
5       unifo